# Dominio applicativo
il dominio prevede 4 classi: [Mazzo] [Partita] [Giocatore] [Sarsa]

In [51]:
from random import randint, random, shuffle
from time import time, sleep
import pickle
import os

# Mazzo

In [52]:
class Mazzo():
    # mazzo di carte, ogni carta assume valore tra 0 a 39
    # le decine rappresentano i semi
    # le unità rappresentano i valori
    def __init__(self):
        self.carte = [i for i in range(0,40)]
        # si randomizza il mazzo
        shuffle(self.carte)

    def carteRimaste(self):
        return len(self.carte)
    
    def getUltimaCarta(self):
        return self.carte[-1]

    def pesca(self):
        return self.carte.pop(0)

    def reset(self):
        self.__init__()

# Giocatore

In [53]:
class Giocatore():

    def __init__(self, tipo = 0):
        self.mano = []
        self.punti = 0
        # tipo indica la modalità con cui viene scleta la mossa del giocatore
        # tipo 0 -> giocatore post-apprendimento
        # tipo 1 -> giocaore apprendimento 
        # tipo 2 -> giocatore casuale
        self.tipo = tipo
    
    def reset(self):
        self.mano = []
        self.punti = 0

    def pesca(self, mazzo):
        self.mano.append(mazzo.pesca())
    
    # restituisce un intero tra quelli presenti nella mano
    def gioca(self, giocataAvversario = None):
        match self.tipo:
            case 0: return self.mossaOttimale(giocataAvversario)
            case 1: return self.apprendi(giocataAvversario)
            case 2: return self.mossaCasuale()
        
    def mossaOttimale(self, giocataAvversario):
        # gestione della mossa scelta tramite la policy ottima
        return

    def apprendi(self, giocataAvversario):
        # gestione apprendimento
        return
    
    def mossaCasuale(self):
        n = randint(0, 2)
        return self.mano.pop(n)

    def setPunti(self, punti):
        self.punti += punti
    
    def getPunti(self):
        return self.punti

    # gestisci reward
    def setReward(self, reward):
        # TO_DO
        return

# Sarsa

In [ ]:
class Sarsa():

    def __init__(self):
        pass

# Gioco

In [54]:
class Gioco():
    
    WIN_REWARD = 200

    # tipo indica il tipo dei giocatori che giocano la partita
    # tipo 0 -> giocatore post-apprendimento
    # tipo 1 -> giocaore apprendimento 
    # tipo 2 -> giocatore casuale
    def __init__(self, tipo1 = 0, tipo2 = 0):
        self.giocatori = (Giocatore(tipo1), Giocatore(tipo2))
        self.vittorie = (0,0)
        self.mazzo = Mazzo()
        self.briscola = self.mazzo.getUltimaCarta()
        self.diTurno = 0
    
    def reset(self):
        for i in range(0,1):
            self.giocatori[i].reset()
        
        self.mazzo.reset()
        # si alterna il giocatore che inizia
        self.diTurno = (self.diTurno + 1) % 2 

    def pesca(self):
        self.giocatori[self.diTurno].pesca(self.mazzo)
        self.giocatori[(self.diTurno + 1) % 2]
    
    def partita(self):
        # i giocatori pescano 3 carte
        for _ in range(0, 3):
            self.giocatori[self.diTurno].pesca(self.mazzo)
            self.giocatori[(self.diTurno + 1) % 2].pesca(self.mazzo)

        while(self.mazzo.carteRimaste() > 0):
            self.turno()

            # controllo la vittoria o il fine partita
            if(self.giocatori[0].getPunti() > 60): 
                self.vittorie = (self.vittorie[0]+1, self.vittorie[1])
                self.giocatori[0].setReward(self.WIN_REWARD)
                break

            if(self.giocatori[1].getPunti() > 60): 
                self.vittorie = (self.vittorie[0], self.vittorie[1]+1)
                self.giocatori[0].setReward(self.WIN_REWARD)
                break

        self.reset()
        # a scopo di debug
        print(self.vittorie) 
    
    def turno(self):
        # il secondo giocatore conosce la giocata del primo
        primaGiocata = self.giocatori[self.diTurno].gioca()
        giocate = (primaGiocata, self.giocatori[(self.diTurno + 1) % 2].gioca(primaGiocata))
        
        # valutazione = (vincitore, punti)
        valutazione = self.valutaGiocate(giocate)

        # aggiorno il giocatore di turno
        self.diTurno = valutazione[0]

        # assegno punti e reward ai giocatori
        self.giocatori[valutazione[0]].setPunti(valutazione[1])
        self.giocatori[valutazione[0]].setReward(valutazione[1])
        self.giocatori[(valutazione[0] + 1) % 2].setReward(-valutazione[1])

        # ridistribuisco le carte
        self.giocatori[self.diTurno].pesca(self.mazzo)
        self.giocatori[(self.diTurno + 1) % 2].pesca(self.mazzo)      

    # restituisce la tupla (vincitore, punti)   
    def valutaGiocate(self, giocate):
        semi = (giocate[0]//10, giocate[1]//10)
        punti = (self.calcolaPunti(giocate[0]), self.calcolaPunti(giocate[1]))

        if semi[0] == semi[1]:
            # 0 -> giocatore di turno
            if giocate[0] > giocate[1]: return (self.diTurno, punti[0] + punti[1])
            return ((self.diTurno + 1) % 2, punti[0] + punti[1])

        if semi[1] == self.briscola//10: return ((self.diTurno + 1) % 2, punti[0] + punti[1])
        return (self.diTurno, punti[0] + punti[1])

    def calcolaPunti(self, giocata):
        valore = giocata%10 + 1
        if valore == 1: return 11
        if valore == 3: return 10
        if valore < 8: return 0
        return valore - 6

In [59]:
gioco = Gioco(2, 2)
gioco.partita()

(0, 1)
